In [3]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta
import io

try:
    import polars as pl
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "polars"], check=True)
    import polars as pl

print("🚀 Advanced Polars Analytics Pipeline")
print("=" * 50)

🚀 Advanced Polars Analytics Pipeline


In [4]:
np.random.seed(42)
n_records = 100000
dates = [datetime(2020, 1, 1) + timedelta(days=i//100) for i in range(n_records)]
tickers = np.random.choice(['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN'], n_records)

# Create complex synthetic dataset
data = {
    'timestamp': dates,
    'ticker': tickers,
    'price': np.random.lognormal(4, 0.3, n_records),
    'volume': np.random.exponential(1000000, n_records).astype(int),
    'bid_ask_spread': np.random.exponential(0.01, n_records),
    'market_cap': np.random.lognormal(25, 1, n_records),
    'sector': np.random.choice(['Tech', 'Finance', 'Healthcare', 'Energy'], n_records)
}

print(f"📊 Generated {n_records:,} synthetic financial records")

📊 Generated 100,000 synthetic financial records


In [5]:
lf = pl.LazyFrame(data)

result = (
    lf
    .with_columns([
        pl.col('timestamp').dt.year().alias('year'),
        pl.col('timestamp').dt.month().alias('month'),
        pl.col('timestamp').dt.weekday().alias('weekday'),
        pl.col('timestamp').dt.quarter().alias('quarter')
    ])

    .with_columns([
        pl.col('price').rolling_mean(20).over('ticker').alias('sma_20'),
        pl.col('price').rolling_std(20).over('ticker').alias('volatility_20'),

        pl.col('price').ewm_mean(span=12).over('ticker').alias('ema_12'),

        pl.col('price').diff().alias('price_diff'),

        (pl.col('volume') * pl.col('price')).alias('dollar_volume')
    ])

    .with_columns([
        pl.col('price_diff').clip(0, None).rolling_mean(14).over('ticker').alias('rsi_up'),
        pl.col('price_diff').abs().rolling_mean(14).over('ticker').alias('rsi_down'),

        (pl.col('price') - pl.col('sma_20')).alias('bb_position')
    ])

    .with_columns([
        (100 - (100 / (1 + pl.col('rsi_up') / pl.col('rsi_down')))).alias('rsi')
    ])

    .filter(
        (pl.col('price') > 10) &
        (pl.col('volume') > 100000) &
        (pl.col('sma_20').is_not_null())
    )

    .group_by(['ticker', 'year', 'quarter'])
    .agg([
        pl.col('price').mean().alias('avg_price'),
        pl.col('price').std().alias('price_volatility'),
        pl.col('price').min().alias('min_price'),
        pl.col('price').max().alias('max_price'),
        pl.col('price').quantile(0.5).alias('median_price'),

        pl.col('volume').sum().alias('total_volume'),
        pl.col('dollar_volume').sum().alias('total_dollar_volume'),

        pl.col('rsi').filter(pl.col('rsi').is_not_null()).mean().alias('avg_rsi'),
        pl.col('volatility_20').mean().alias('avg_volatility'),
        pl.col('bb_position').std().alias('bollinger_deviation'),

        pl.len().alias('trading_days'),
        pl.col('sector').n_unique().alias('sectors_count'),

        (pl.col('price') > pl.col('sma_20')).mean().alias('above_sma_ratio'),

        ((pl.col('price').max() - pl.col('price').min()) / pl.col('price').min())
          .alias('price_range_pct')
    ])

    .with_columns([
        pl.col('total_dollar_volume').rank(method='ordinal', descending=True).alias('volume_rank'),
        pl.col('price_volatility').rank(method='ordinal', descending=True).alias('volatility_rank')
    ])

    .filter(pl.col('trading_days') >= 10)
    .sort(['ticker', 'year', 'quarter'])
)

In [6]:
df = result.collect()
print(f"\n📈 Analysis Results: {df.height:,} aggregated records")
print("\nTop 10 High-Volume Quarters:")
print(df.sort('total_dollar_volume', descending=True).head(10).to_pandas())

print("\n🔍 Advanced Analytics:")

pivot_analysis = (
    df.group_by('ticker')
    .agg([
        pl.col('avg_price').mean().alias('overall_avg_price'),
        pl.col('price_volatility').mean().alias('overall_volatility'),
        pl.col('total_dollar_volume').sum().alias('lifetime_volume'),
        pl.col('above_sma_ratio').mean().alias('momentum_score'),
        pl.col('price_range_pct').mean().alias('avg_range_pct')
    ])
    .with_columns([
        (pl.col('overall_avg_price') / pl.col('overall_volatility')).alias('risk_adj_score'),

        (pl.col('momentum_score') * 0.4 +
         pl.col('avg_range_pct') * 0.3 +
         (pl.col('lifetime_volume') / pl.col('lifetime_volume').max()) * 0.3)
         .alias('composite_score')
    ])
    .sort('composite_score', descending=True)
)

print("\n🏆 Ticker Performance Ranking:")
print(pivot_analysis.to_pandas())


📈 Analysis Results: 55 aggregated records

Top 10 High-Volume Quarters:
  ticker  year  quarter  avg_price  price_volatility  min_price   max_price  \
0  GOOGL  2021        1  57.202767         17.959219  21.381671  178.847677   
1  GOOGL  2021        4  58.237738         18.053261  21.166928  141.119185   
2   TSLA  2020        3  56.841917         17.185146  21.578360  147.829411   
3   AMZN  2020        2  57.284240         17.566920  17.889955  145.269024   
4   AMZN  2021        2  56.966529         17.507191  20.161137  157.495422   
5   AMZN  2021        3  56.952561         17.064709  22.396973  137.289107   
6   AMZN  2020        4  56.963474         17.984251  18.077893  154.224561   
7  GOOGL  2020        4  57.423558         17.367769  22.837007  132.253255   
8  GOOGL  2020        1  57.198658         17.192811  19.157802  160.768177   
9   TSLA  2021        1  56.670237         17.413646  19.237795  151.668559   

   median_price  total_volume  total_dollar_volume    avg

In [7]:
print("\n🔄 SQL Interface Demo:")
pl.Config.set_tbl_rows(5)

sql_result = pl.sql("""
    SELECT
        ticker,
        AVG(avg_price) as mean_price,
        STDDEV(price_volatility) as volatility_consistency,
        SUM(total_dollar_volume) as total_volume,
        COUNT(*) as quarters_tracked
    FROM df
    WHERE year >= 2021
    GROUP BY ticker
    ORDER BY total_volume DESC
""", eager=True)

print(sql_result)

print(f"\n⚡ Performance Metrics:")
print(f"   • Lazy evaluation optimizations applied")
print(f"   • {n_records:,} records processed efficiently")
print(f"   • Memory-efficient columnar operations")
print(f"   • Zero-copy operations where possible")

print(f"\n💾 Export Options:")
print("   • Parquet (high compression): df.write_parquet('data.parquet')")
print("   • Delta Lake: df.write_delta('delta_table')")
print("   • JSON streaming: df.write_ndjson('data.jsonl')")
print("   • Apache Arrow: df.to_arrow()")

print("\n✅ Advanced Polars pipeline completed successfully!")
print("🎯 Demonstrated: Lazy evaluation, complex expressions, window functions,")
print("   SQL interface, advanced aggregations, and high-performance analytics")


🔄 SQL Interface Demo:
shape: (5, 5)
┌────────┬────────────┬────────────────────────┬──────────────┬──────────────────┐
│ ticker ┆ mean_price ┆ volatility_consistency ┆ total_volume ┆ quarters_tracked │
│ ---    ┆ ---        ┆ ---                    ┆ ---          ┆ ---              │
│ str    ┆ f64        ┆ f64                    ┆ f64          ┆ u32              │
╞════════╪════════════╪════════════════════════╪══════════════╪══════════════════╡
│ GOOGL  ┆ 57.196227  ┆ 0.448066               ┆ 7.3922e11    ┆ 7                │
│ AMZN   ┆ 56.951047  ┆ 0.266619               ┆ 7.2531e11    ┆ 7                │
│ TSLA   ┆ 57.172448  ┆ 0.337545               ┆ 7.2022e11    ┆ 7                │
│ MSFT   ┆ 57.258385  ┆ 0.447526               ┆ 7.1621e11    ┆ 7                │
│ AAPL   ┆ 56.988095  ┆ 0.47604                ┆ 7.1573e11    ┆ 7                │
└────────┴────────────┴────────────────────────┴──────────────┴──────────────────┘

⚡ Performance Metrics:
   • Lazy evaluation optim